# Getting transcripts from Ukrainian government website

Run the following command in the terminal to install the versions we use in this notebook:

    pip install pandas=='1.3.3' beautifulsoup4=='4.11.1' selenium=='4.3.0' webdriver_manager=='3.7.1' tqdm=='4.62.2'

Importing packages

In [1]:
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/maiapolo/.wdm/drivers/chromedriver/mac64/103.0.5060.134/chromedriver] found in cache
/usr/local/anaconda/envs/stats600/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


Getting texts URLs:

In [2]:
links = []

for i in tqdm(range(1,29)):
    
    driver.get("https://www.president.gov.ua/en/news/speeches?page="+str(i))

    source=str(driver.page_source)
    txt = source
    ind, ind2, ind3 = 0, 0, 0

    while True:
        txt=txt[(ind+ind2+ind3):]

        ind=txt.find('item_stat_headline')
        ind2=txt[ind:].find('https://')
        ind3=txt[(ind+ind2):].find('">')

        links.append([txt[(ind+ind2):(ind+ind2+ind3)]])

        if links[-1]==['']: 
            links=links[:-1]
            break

100%|██████████| 28/28 [00:31<00:00,  1.12s/it]


In [3]:
len(links)

273

Getting texts:

In [4]:
for i in tqdm(range(len(links))):

    driver.get(links[i][0])

    source=str(driver.page_source)
    txt = source
    
    ind=txt.rfind('"date"')+7
    ind2=txt[ind:].find('</p>')
    date=txt[ind:(ind+ind2)]
    date=" ".join(date.split())

    ind3=source[(ind+ind2):].find('title="" alt="')+14
    ind4=source[(ind+ind2+ind3):].find('">')
    title=source[(ind+ind2+ind3):(ind+ind2+ind3+ind4)]

    soup = bs(source[(ind+ind2):], features="html.parser")
    text = soup.get_text()
    ind5=text.rfind('\n\n\n\n\n\nv')
    text = text[:ind5]
    text=text.replace("\n", " ")
    text=" ".join(text.split())

    links[i]+=[date,title,text]

100%|██████████| 273/273 [08:22<00:00,  1.84s/it]


Saving CSV:

In [5]:
data = {'url':[l[0] for l in links], 'date':[l[1] for l in links], 'title':[l[2] for l in links], 'text':[l[3] for l in links]}

In [6]:
data=pd.DataFrame(data)

In [7]:
data.to_csv('data/zelensky_speeches.csv')